# 用户画像推荐算法

In [12]:
import pandas as pd
import numpy as np
import ast
import collections
from functools import reduce

In [2]:
movie_profile = pd.read_csv('movie_profile.csv')
movie_profile

,MOVIE_ID,NAME,PROFILE,WEIGHTS
0,26670818,情定河州,"['临夏', '宣传伊斯兰教的电影', '甘肃', '伊斯兰', '中国大陆', '2016...","{'临夏': 0.5133773014499408, '宣传伊斯兰教的电影': 0.5133..."
1,25815002,我不是李小龙,"['穿越', '华语', '动作', '爱情']","{'穿越': 0.7136470387832085, '华语': 0.63509936398..."
2,26392287,曼哈顿中国女孩,['剧情'],{'剧情': 1.0}
3,26695995,绿毛水怪,"['小波', '王小波', '文学改编', '小说改编', '2017', '爱情', '中...","{'小波': 0.607881377207677, '王小波': 0.55139953487..."
4,26392292,为了祖国,"['剧情', '历史']","{'剧情': 1.0, '历史': 1.0}"
...,...,...,...,...
136447,5108157,Longe da Vista - 电影,['剧情'],{'剧情': 1.0}
136448,5108126,Phera - 电影,"['India', '10', '7', '剧情']","{'India': 0.6037789518421052, '10': 0.56442000..."
136449,5108078,Fast Track - 电影,"['剧情', '动作']","{'剧情': 1.0, '动作': 1.0}"
136450,5108060,7-9-13 - 电影,['喜剧'],{'喜剧': 1.0}


In [3]:
movie_profile['PROFILE'] = movie_profile['PROFILE'].apply(lambda x: ast.literal_eval(x))
movie_profile

,MOVIE_ID,NAME,PROFILE,WEIGHTS
0,26670818,情定河州,"[临夏, 宣传伊斯兰教的电影, 甘肃, 伊斯兰, 中国大陆, 2016, 烂片, 中国, 爱...","{'临夏': 0.5133773014499408, '宣传伊斯兰教的电影': 0.5133..."
1,25815002,我不是李小龙,"[穿越, 华语, 动作, 爱情]","{'穿越': 0.7136470387832085, '华语': 0.63509936398..."
2,26392287,曼哈顿中国女孩,[剧情],{'剧情': 1.0}
3,26695995,绿毛水怪,"[小波, 王小波, 文学改编, 小说改编, 2017, 爱情, 中国, 剧情]","{'小波': 0.607881377207677, '王小波': 0.55139953487..."
4,26392292,为了祖国,"[剧情, 历史]","{'剧情': 1.0, '历史': 1.0}"
...,...,...,...,...
136447,5108157,Longe da Vista - 电影,[剧情],{'剧情': 1.0}
136448,5108126,Phera - 电影,"[India, 10, 7, 剧情]","{'India': 0.6037789518421052, '10': 0.56442000..."
136449,5108078,Fast Track - 电影,"[剧情, 动作]","{'剧情': 1.0, '动作': 1.0}"
136450,5108060,7-9-13 - 电影,[喜剧],{'喜剧': 1.0}


In [4]:
movie_profile['WEIGHTS'] = movie_profile['WEIGHTS'].apply(lambda x: ast.literal_eval(x))
movie_profile

,MOVIE_ID,NAME,PROFILE,WEIGHTS
0,26670818,情定河州,"[临夏, 宣传伊斯兰教的电影, 甘肃, 伊斯兰, 中国大陆, 2016, 烂片, 中国, 爱...","{'临夏': 0.5133773014499408, '宣传伊斯兰教的电影': 0.5133..."
1,25815002,我不是李小龙,"[穿越, 华语, 动作, 爱情]","{'穿越': 0.7136470387832085, '华语': 0.63509936398..."
2,26392287,曼哈顿中国女孩,[剧情],{'剧情': 1.0}
3,26695995,绿毛水怪,"[小波, 王小波, 文学改编, 小说改编, 2017, 爱情, 中国, 剧情]","{'小波': 0.607881377207677, '王小波': 0.55139953487..."
4,26392292,为了祖国,"[剧情, 历史]","{'剧情': 1.0, '历史': 1.0}"
...,...,...,...,...
136447,5108157,Longe da Vista - 电影,[剧情],{'剧情': 1.0}
136448,5108126,Phera - 电影,"[India, 10, 7, 剧情]","{'India': 0.6037789518421052, '10': 0.56442000..."
136449,5108078,Fast Track - 电影,"[剧情, 动作]","{'剧情': 1.0, '动作': 1.0}"
136450,5108060,7-9-13 - 电影,[喜剧],{'喜剧': 1.0}


In [5]:
movie_profile.set_index('MOVIE_ID', inplace=True)
movie_profile

,NAME,PROFILE,WEIGHTS
MOVIE_ID,,,
26670818,情定河州,"[临夏, 宣传伊斯兰教的电影, 甘肃, 伊斯兰, 中国大陆, 2016, 烂片, 中国, 爱...","{'临夏': 0.5133773014499408, '宣传伊斯兰教的电影': 0.5133..."
25815002,我不是李小龙,"[穿越, 华语, 动作, 爱情]","{'穿越': 0.7136470387832085, '华语': 0.63509936398..."
26392287,曼哈顿中国女孩,[剧情],{'剧情': 1.0}
26695995,绿毛水怪,"[小波, 王小波, 文学改编, 小说改编, 2017, 爱情, 中国, 剧情]","{'小波': 0.607881377207677, '王小波': 0.55139953487..."
26392292,为了祖国,"[剧情, 历史]","{'剧情': 1.0, '历史': 1.0}"
...,...,...,...
5108157,Longe da Vista - 电影,[剧情],{'剧情': 1.0}
5108126,Phera - 电影,"[India, 10, 7, 剧情]","{'India': 0.6037789518421052, '10': 0.56442000..."
5108078,Fast Track - 电影,"[剧情, 动作]","{'剧情': 1.0, '动作': 1.0}"


In [6]:
inverted_table = pd.read_csv('inverted_table.csv', names=['TAG', 'list(MOVIE_ID, WEIGHT)'])
inverted_table

,TAG,"list(MOVIE_ID, WEIGHT)"
0,临夏,"[(26670818, 0.5133773014499408)]"
1,宣传伊斯兰教的电影,"[(26670818, 0.5133773014499408)]"
2,甘肃,"[(26670818, 0.43558046555429536), (27017988, 0..."
3,伊斯兰,"[(26670818, 0.4054845411941), (20439410, 0.903..."
4,中国大陆,"[(26670818, 0.19552557830601686), (26996825, 0..."
...,...,...
64863,玛雅诅咒,"[(5109239, 0.6543921207159601)]"
64864,赤座美代子,"[(5107867, 0.5764199843225035)]"
64865,长篇幅,"[(5108441, 0.9457279343197208)]"
64866,PBM,"[(5108179, 0.5794429088709652)]"


In [7]:
inverted_table['list(MOVIE_ID, WEIGHT)'] = inverted_table['list(MOVIE_ID, WEIGHT)'].apply(lambda x: ast.literal_eval(x))
inverted_table

,TAG,"list(MOVIE_ID, WEIGHT)"
0,临夏,"[(26670818, 0.5133773014499408)]"
1,宣传伊斯兰教的电影,"[(26670818, 0.5133773014499408)]"
2,甘肃,"[(26670818, 0.43558046555429536), (27017988, 0..."
3,伊斯兰,"[(26670818, 0.4054845411941), (20439410, 0.903..."
4,中国大陆,"[(26670818, 0.19552557830601686), (26996825, 0..."
...,...,...
64863,玛雅诅咒,"[(5109239, 0.6543921207159601)]"
64864,赤座美代子,"[(5107867, 0.5764199843225035)]"
64865,长篇幅,"[(5108441, 0.9457279343197208)]"
64866,PBM,"[(5108179, 0.5794429088709652)]"


In [8]:
inverted_table.set_index('TAG', inplace=True)
inverted_table

,"list(MOVIE_ID, WEIGHT)"
TAG,
临夏,"[(26670818, 0.5133773014499408)]"
宣传伊斯兰教的电影,"[(26670818, 0.5133773014499408)]"
甘肃,"[(26670818, 0.43558046555429536), (27017988, 0..."
伊斯兰,"[(26670818, 0.4054845411941), (20439410, 0.903..."
中国大陆,"[(26670818, 0.19552557830601686), (26996825, 0..."
...,...
玛雅诅咒,"[(5109239, 0.6543921207159601)]"
赤座美代子,"[(5107867, 0.5764199843225035)]"
长篇幅,"[(5108441, 0.9457279343197208)]"


In [9]:
watch_record = pd.read_csv('watch_record.csv')
watch_record

,USER_MD5,MOVIE_ID
0,00000d431c3ef737f7a2cc7d3cf2540a,"[2189725, 1458424, 4221515, 3070921, 6815094, ..."
1,00008ef628c42d8089253c0906804c7e,"[26972258, 27050878, 30140107, 1301771, 266332..."
2,0000907ad9784ed6c758550e6e443e4d,"[11594934, 6974521, 4078562]"
3,00009ed66bc8a8a867b7887ccedabba4,[1458862]
4,0000d9d279b603cc4c1914eda90b62a2,"[3096161, 1310178]"
...,...,...
600379,ffff5bb919b23a534bf9858f64d1d4b7,"[1482354, 1305419, 3105162, 2255841]"
600380,ffff66342d284e19b75ab76390268d74,"[3041772, 1777200, 4016479, 1307773]"
600381,ffff875dc090c1109b3d37ce5b114075,[1305814]
600382,ffff9498af27bdb06c3e61452758fba5,"[10566533, 5294307, 26265987, 5308199, 6080698..."


In [10]:
watch_record['MOVIE_ID'] = watch_record['MOVIE_ID'].apply(lambda x: ast.literal_eval(x))
watch_record

,USER_MD5,MOVIE_ID
0,00000d431c3ef737f7a2cc7d3cf2540a,"[2189725, 1458424, 4221515, 3070921, 6815094, ..."
1,00008ef628c42d8089253c0906804c7e,"[26972258, 27050878, 30140107, 1301771, 266332..."
2,0000907ad9784ed6c758550e6e443e4d,"[11594934, 6974521, 4078562]"
3,00009ed66bc8a8a867b7887ccedabba4,[1458862]
4,0000d9d279b603cc4c1914eda90b62a2,"[3096161, 1310178]"
...,...,...
600379,ffff5bb919b23a534bf9858f64d1d4b7,"[1482354, 1305419, 3105162, 2255841]"
600380,ffff66342d284e19b75ab76390268d74,"[3041772, 1777200, 4016479, 1307773]"
600381,ffff875dc090c1109b3d37ce5b114075,[1305814]
600382,ffff9498af27bdb06c3e61452758fba5,"[10566533, 5294307, 26265987, 5308199, 6080698..."


In [11]:
watch_record.set_index('USER_MD5', inplace=True)
watch_record

,MOVIE_ID
USER_MD5,
00000d431c3ef737f7a2cc7d3cf2540a,"[2189725, 1458424, 4221515, 3070921, 6815094, ..."
00008ef628c42d8089253c0906804c7e,"[26972258, 27050878, 30140107, 1301771, 266332..."
0000907ad9784ed6c758550e6e443e4d,"[11594934, 6974521, 4078562]"
00009ed66bc8a8a867b7887ccedabba4,[1458862]
0000d9d279b603cc4c1914eda90b62a2,"[3096161, 1310178]"
...,...
ffff5bb919b23a534bf9858f64d1d4b7,"[1482354, 1305419, 3105162, 2255841]"
ffff66342d284e19b75ab76390268d74,"[3041772, 1777200, 4016479, 1307773]"
ffff875dc090c1109b3d37ce5b114075,[1305814]


In [15]:
len(watch_record.index)

600384

In [16]:
userprofile_results = {}

In [19]:
for i in range(1000):
    user_md5 = watch_record.index[i]
    watched_movies = watch_record.loc[user_md5, 'MOVIE_ID']
    record_movie_profile = movie_profile.loc[list(set(watched_movies) & set(movie_profile.index))]
    if len(record_movie_profile) == 0:
        userprofile_results[user_md5] = []
        continue
    counter = collections.Counter(reduce(lambda x, y: list(x) + list(y), record_movie_profile['PROFILE'].values))
    interest_words = counter.most_common(50)
    maxcount = interest_words[0][1]
    interest_words = [(w, round(c / maxcount, 4)) for w, c in interest_words]
    result_table = {}
    for interest_word, interest_weight in interest_words:
        related_movies = inverted_table.loc[interest_word, 'list(MOVIE_ID, WEIGHT)']
        for mid, related_weight in related_movies:
            if mid not in watched_movies:
                _ = result_table.get(mid, [])
                _.append(interest_weight * related_weight)
                result_table.setdefault(mid, _)
    rs_result = map(lambda x: (x[0], sum(x[1])), result_table.items())
    rs_result = sorted(rs_result, key=lambda x: x[1], reverse=True)[:12]
    recommed_movies = [x[0] for x in rs_result]
    userprofile_results[user_md5] = recommed_movies

In [21]:
userprofile_results

{'00000d431c3ef737f7a2cc7d3cf2540a': [5159690,
  1309132,
  1307358,
  26696072,
  1830411,
  25944714,
  4915058,
  1294419,
  3428547,
  3022436,
  1422915,
  2139010],
 '00008ef628c42d8089253c0906804c7e': [1298913,
  5098322,
  26820466,
  25757376,
  25935962,
  25943189,
  27005771,
  26067589,
  26889753,
  25975192,
  25983932,
  25767344],
 '0000907ad9784ed6c758550e6e443e4d': [20378834,
  6080775,
  26995429,
  25711540,
  1456189,
  6402119,
  20501058,
  3041773,
  11615906,
  1358423,
  1308118,
  1304985],
 '00009ed66bc8a8a867b7887ccedabba4': [26853439,
  1305355,
  1306951,
  1303176,
  1297223,
  2003850,
  1302531,
  1308827,
  3804746,
  3182495,
  26877389,
  11523469],
 '0000d9d279b603cc4c1914eda90b62a2': [1299800,
  1293369,
  1296494,
  1304985,
  1298897,
  26982978,
  2269064,
  1298410,
  1295988,
  1306875,
  5028431,
  1297756],
 '0000ff6ecbaeada120792d2a575d26a1': [3531073,
  1979015,
  1463710,
  1296936,
  1300115,
  24743765,
  1302205,
  5109727,
  1469492

In [22]:
import json

In [23]:
with open('userprofile_results.json', 'a') as file:
    file.write(json.dumps(userprofile_results))

In [51]:
watched_movies = watch_record.loc[user_md5, 'MOVIE_ID']
watched_movies

[1458862]

In [52]:
record_movie_profile = movie_profile.loc[list(set(watched_movies) & set(movie_profile.index))]
record_movie_profile

,NAME,PROFILE,WEIGHTS
MOVIE_ID,,,
1458862,妃子笑,"[妃子笑, 2005, 古装, 烂片, 喜剧, 香港电影, 香港, 电影]","{'妃子笑': 0.7634245085929579, '2005': 0.34269245..."


In [53]:
import collections

In [54]:
from functools import reduce

In [55]:
counter = collections.Counter(reduce(lambda x, y: list(x) + list(y), record_movie_profile['PROFILE'].values))
counter

Counter({'妃子笑': 1,
         '2005': 1,
         '古装': 1,
         '烂片': 1,
         '喜剧': 1,
         '香港电影': 1,
         '香港': 1,
         '电影': 1})

In [56]:
interest_words = counter.most_common(50)
interest_words

[('妃子笑', 1),
 ('2005', 1),
 ('古装', 1),
 ('烂片', 1),
 ('喜剧', 1),
 ('香港电影', 1),
 ('香港', 1),
 ('电影', 1)]

In [57]:
maxcount = interest_words[0][1]
maxcount

1

In [58]:
_interest_words = [(w, round(c / maxcount, 4)) for w, c in interest_words]
_interest_words

[('妃子笑', 1.0),
 ('2005', 1.0),
 ('古装', 1.0),
 ('烂片', 1.0),
 ('喜剧', 1.0),
 ('香港电影', 1.0),
 ('香港', 1.0),
 ('电影', 1.0)]

In [59]:
user_profile = interest_words

In [64]:
result_table = {}
for interest_word, interest_weight in interest_words:
    related_movies = inverted_table.loc[interest_word, 'list(MOVIE_ID, WEIGHT)']
    for mid, related_weight in related_movies:
        if mid not in watched_movies:
            _ = result_table.get(mid, [])
            _.append(interest_weight * related_weight)
            result_table.setdefault(mid, _)
result_table

{26734718: [0.28544477147714736],
 2991446: [0.3055290846950545, 1.0],
 2968938: [0.29289589201665606],
 1866832: [0.29152497876689326, 0.1250044644451285],
 1946303: [0.2534147442566401],
 1467246: [0.3357832896671781, 0.1439822085641322],
 1421885: [0.2789846156403667, 1.0],
 1963133: [0.2581400026678059],
 1441293: [0.41231280922878166],
 1422963: [0.2879524381753962],
 2997471: [0.26366423403242],
 2330341: [0.27290403357643406, 0.1880043734186597, 0.11701989553840471],
 25869944: [0.8382575670880168, 1.0],
 1441413: [0.30682139645365414, 0.13156349245346766],
 1422949: [0.2669175408219287, 1.0],
 3016704: [0.274234107631376],
 1424218: [0.24982092852346535],
 1422955: [0.3909224626418923],
 3023355: [0.3650101522314516,
  1.0,
  0.23691137155981506,
  0.21899208211497165,
  0.1565145422177083],
 4217483: [0.25427764706758366],
 1464901: [0.32311923746756244, 0.2097218974809811, 0.19385914104535493],
 1870121: [0.3047027534069051, 0.1306550288270248],
 1438280: [0.3227141540151444]

In [65]:
rs_result = map(lambda x: (x[0], sum(x[1])), result_table.items())
rs_result = sorted(rs_result, key=lambda x: x[1], reverse=True)[:100]
rs_result

[(26853439, 2.5549231684316123),
 (1305355, 2.547401272973748),
 (1306951, 2.5445359884711887),
 (1303176, 2.5138844002213587),
 (1297223, 2.4857151928342183),
 (2003850, 2.4852208846240935),
 (1302531, 2.4803803992254636),
 (1308827, 2.4741082225772337),
 (3804746, 2.466990632694289),
 (3182495, 2.465520126662847),
 (26877389, 2.4375007342580903),
 (11523469, 2.4340946001932457),
 (4735944, 2.424365853755692),
 (30416337, 2.4190299484961404),
 (3670447, 2.414992096075279),
 (1306381, 2.4084324898714042),
 (30331264, 2.4078377225561325),
 (4913222, 2.3931615204841425),
 (1307637, 2.3705202505536276),
 (1767732, 2.3655449458059725),
 (26614144, 2.3646176622982047),
 (1303497, 2.364036798936197),
 (1824534, 2.3608597348533515),
 (1308111, 2.359093984307346),
 (1303193, 2.3584470189270696),
 (4707596, 2.357848328605606),
 (1302654, 2.3562325042031786),
 (1305060, 2.3542278361341524),
 (1305265, 2.3518401827556925),
 (3076681, 2.3495792509483797),
 (1297726, 2.348167936446246),
 (1306249, 

In [66]:
recommed_movies = [x[0] for x in rs_result]
recommed_movies

[26853439,
 1305355,
 1306951,
 1303176,
 1297223,
 2003850,
 1302531,
 1308827,
 3804746,
 3182495,
 26877389,
 11523469,
 4735944,
 30416337,
 3670447,
 1306381,
 30331264,
 4913222,
 1307637,
 1767732,
 26614144,
 1303497,
 1824534,
 1308111,
 1303193,
 4707596,
 1302654,
 1305060,
 1305265,
 3076681,
 1297726,
 1306249,
 1871997,
 1301982,
 1416728,
 1307323,
 1293518,
 1297763,
 1301213,
 1295987,
 1481837,
 1294488,
 32494059,
 1937380,
 1301116,
 30206502,
 2090739,
 6710802,
 1461031,
 26668583,
 1298761,
 26794472,
 1301018,
 25824856,
 1408298,
 1302454,
 26329802,
 26324929,
 26919705,
 26821322,
 24879839,
 27173200,
 30128900,
 30280011,
 26895377,
 1292213,
 30269047,
 1295485,
 5178517,
 27041538,
 10573695,
 30156320,
 3993553,
 30287947,
 33427447,
 1776365,
 30128899,
 1299398,
 1464602,
 4745484,
 27097908,
 3928370,
 1297863,
 26600240,
 1316510,
 26769600,
 1294561,
 30242180,
 27069429,
 27205169,
 10521678,
 4822848,
 6965647,
 26728186,
 2347268,
 6905175,
 2683

In [67]:
recommend_movies_info = movies.loc[recommed_movies, :]
recommend_movies_info

,NAME,ALIAS,ACTORS,COVER,DIRECTORS,DOUBAN_SCORE,DOUBAN_VOTES,GENRES,IMDB_ID,LANGUAGES,MINS,OFFICIAL_SITE,REGIONS,RELEASE_DATE,SLUG,STORYLINE,TAGS,YEAR,ACTOR_IDS,DIRECTOR_IDS
MOVIE_ID,,,,,,,,,,,,,,,,,,,,
26853439,谁杀了潘巾莲2,谁杀了潘巾莲2白衣神探,季子渊/简佳琳/艺心/汤家宽/徐锦明/燕红兵/宋锐贤/胡凯莉/许颖鹏,NaN,凌学松,0.0,0.0,剧情/喜剧/悬疑/古装,NaN,汉语普通话,65.0,NaN,中国大陆,2016-08-10,2jbiVfEZf,三年前，秦公子因一桩离奇凶案眼睛失明，得到郓哥的解救后，后与郓哥摆摊算卦为生，潜伏于表面平静...,烂片,2016.0,季子渊:|简佳琳:1352779|艺心:|汤家宽:|徐锦明:|燕红兵:|宋锐贤:|胡凯莉:1...,凌学松:
1305355,审死官,威龙闯天关(台),周星驰/梅艳芳/吴家丽/吴孟达/秦沛/梁家仁/苑琼丹/黄一飞/朱咪咪,NaN,杜琪峰,7.6,122936.0,喜剧/古装,tt0105385,粤语,102.0,NaN,中国香港,1992-07-02,YvqyFRuyb,宋世杰（周星驰 饰）是一个清朝时期的著名大状，他出色的口才及能力让他敢于批评衙门的大官，也为...,周星驰/喜剧/香港/香港电影/梅艳芳/杜琪峰/搞笑/电影,1992.0,周星驰:1048026|梅艳芳:1047976|吴家丽:1274241|吴孟达:101677...,杜琪峰:1160186
1306951,鹿鼎记2：神龙教,NaN,周星驰/邱淑贞/林青霞/李嘉欣/陈百祥/刘松仁/温兆伦/汤镇业/吴君如/袁洁莹/陈德容/秦沛,NaN,王晶,7.8,125752.0,喜剧/动作/武侠/古装,tt0104771,粤语,102.0,NaN,中国香港,1992-09-24,ry7IfU3Ne,韦小宝（周星驰 饰）要护送建宁公主（邱淑贞 饰）嫁到云南，在途中遇到了独臂神尼的刺杀。小宝被...,周星驰/喜剧/香港/香港电影/林青霞/搞笑/王晶/电影,1992.0,周星驰:1048026|邱淑贞:1048296|林青霞:1044746|李嘉欣:113954...,王晶:1274331
1303176,九星报喜,NaN,张国荣/吴倩莲/钟镇涛/李蕙敏/张达明/黎姿/黄百鸣/陈洁灵/周文健/雷宇扬,https://img3.doubanio.com/view/photo/s_ratio_p...,高志森,6.9,8099.0,喜剧/爱情/古装,tt0237501,粤语,86.0,NaN,中国香港,1998-02-14,AAeqaUZnb,彩虹村本是民风淳朴的和谐之村，谁知新上任的知县却见不得村民们互助友爱，认为人性本恶，于是搞得...,张国荣/喜剧/香港/香港电影/1998/高志森/哥哥/电影,1998.0,张国荣:1003494|吴倩莲:1166703|钟镇涛:1033223|李蕙敏:110381...,高志森:1281025
1297223,鹿鼎记,NaN,周星驰/吴孟达/张敏/吴君如/邱淑贞/刘松仁/陈百祥/袁洁莹/温兆伦/徐锦江/陈德容,NaN,王晶,8.0,189917.0,喜剧/动作/武侠/古装,tt0104770,粤语,110.0,NaN,中国香港,1992-07-30,ybqyraaib,清朝初期，康熙皇帝尚且年少，奸臣从中控制政权，民不聊生。“天地会”是一个推翻清朝的民间组织。...,周星驰/喜剧/香港/香港电影/搞笑/王晶/鹿鼎记/电影,1992.0,周星驰:1048026|吴孟达:1016771|张敏:1005297|吴君如:1127819...,王晶:1274331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6905175,石塘咀大学宿舍,NaN,吴启华/黄振辉/陈国邦/胡越山,NaN,卢芬,0.0,0.0,喜剧,NaN,粤语,124.0,NaN,中国香港,2006-10-31,rUZZfaa6e,OK(陈国邦)是一大学三年级学生，因时常逃课去做兼职而导致要留级，后来被大学教授(吴启华)发...,香港电影/香港/喜剧,2006.0,吴启华:|黄振辉:|陈国邦:|胡越山:,卢芬:
26831255,仙医神厨,NaN,王厦/刘艾妮/李国麟/曾伟权/鲁振顺/李雨阳/韩马利/杜燕歌,NaN,董伟,3.7,137.0,喜剧/爱情/古装,NaN,汉语普通话,90.0,NaN,中国大陆,2016-07-11,aERA2VZva,郁郁不得志的演员杨一恒在其父的帮助下取得了一次在剧组跑龙套的机会，在过程中因为对业余剧组的不...,喜剧/2016/大陆/电影/不能算电影/内地/中国/网络电影,2016.0,王厦:1348169|刘艾妮:|李国麟:1274575|曾伟权:1315314|鲁振顺:13...,董伟:1371177
27083732,这个大王我罩的：真命间谍,NaN,苑乔/何索,NaN,张馨元,0.0,0.0,喜剧/爱情/奇幻/古装,NaN,汉语普通话,91.0,NaN,中国大陆,2017-09-12,2FBAIBInb,《这个大王我罩的》讲述的是一个爱玩英雄联盟的大学生不小心穿越到了几千年前的芃国，并且魂穿到了...,喜剧/中国/爱情/穿越/大陆/奇幻/电影/苑乔,2017.0,苑乔:|何索:,张馨元:
